In [2]:
#For what seems like ease of use, I am using Google Colab rather than GitHub CodeSpaces
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
#Of course not everything can be simple so I am working on uploading a local folder of images to Google Colab
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mdwaquarazam/agricultural-crops-image-classification")

print("Path to dataset files:", path)

100%|██████████| 79.0M/79.0M [00:05<00:00, 16.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mdwaquarazam/agricultural-crops-image-classification/versions/1


In [4]:
class PlantDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes

In [5]:
dataset = PlantDataset(data_dir = '/root/.cache/kagglehub/datasets/mdwaquarazam/agricultural-crops-image-classification/versions/1/Agricultural-crops')

In [6]:
len(dataset)
dataset[9]

(<PIL.Image.Image image mode=RGB size=277x182>, 0)

In [7]:
image, label = dataset[9]
image
print(label)

0


In [8]:
data_dir = '/root/.cache/kagglehub/datasets/mdwaquarazam/agricultural-crops-image-classification/versions/1/Agricultural-crops'
target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)

{0: 'Cherry', 1: 'Coffee-plant', 2: 'Cucumber', 3: 'Fox_nut(Makhana)', 4: 'Lemon', 5: 'Olive-tree', 6: 'Pearl_millet(bajra)', 7: 'Tobacco-plant', 8: 'almond', 9: 'banana', 10: 'cardamom', 11: 'chilli', 12: 'clove', 13: 'coconut', 14: 'cotton', 15: 'gram', 16: 'jowar', 17: 'jute', 18: 'maize', 19: 'mustard-oil', 20: 'papaya', 21: 'pineapple', 22: 'rice', 23: 'soyabean', 24: 'sugarcane', 25: 'sunflower', 26: 'tea', 27: 'tomato', 28: 'vigna-radiati(Mung)', 29: 'wheat'}


In [9]:
#Turn the Images into Tensors of the same size
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

data_dir = '/root/.cache/kagglehub/datasets/mdwaquarazam/agricultural-crops-image-classification/versions/1/Agricultural-crops'
dataset = PlantDataset(data_dir, transform)

#Double checking the dimensions of the tensors
image, label = dataset[9]
image.shape

torch.Size([3, 128, 128])

In [10]:
#Iterating over the images in the dataset
for image, label in dataset:
  break

In [11]:
#Batch the datasets into a dataloader for easier/parallel processing
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [12]:
#Iterating over the dataloader
for images, labels in dataloader:
  break
#Checking the dimensions of the dataloader and the labels tensor
images.shape, labels.shape

(torch.Size([32, 3, 128, 128]), torch.Size([32]))

In [17]:
class PlantClassifier(nn.Module):
    def __init__(self, num_classes = 30):
      super(PlantClassifier, self).__init__()
      self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
      self.features = nn.Sequential(*list(self.base_model.children())[:-1])


      enet_out_size = 1280
      self.classifier = nn.Linear(enet_out_size, num_classes)


    def forward(self, x):
      x = self.features(x)
      output = self.classifier(x)
      return output


In [18]:
Model = PlantClassifier(num_classes=30)
print(Model)

PlantClassifier(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_

In [20]:
example = Model(images)
example.shape

torch.Size([32, 30])